In [1]:
!pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)


In [2]:
import os
import json
import pandas as pd

# 3sent 폴더가 포함된 상위 경로 설정
base_dir = r'/home/metaai2/byeonguk_work/I-live-A-commerce/TL1'  # 데이터가 저장된 상위 경로
categories = ['c_event', 'culture', 'enter', 'fm_drama', 'fs_drama', 'history']

data_list = []  # 본문 데이터를 저장할 리스트

# 각 카테고리의 3sent 폴더에서 데이터 추출
for category in categories:
    three_sent_dir = os.path.join(base_dir, category, '20per')  # 3sent 폴더 경로

    # 3sent 폴더의 모든 파일 처리
    for filename in os.listdir(three_sent_dir):
        file_path = os.path.join(three_sent_dir, filename)

        # JSON 파일 읽기
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

            # 본문 데이터 추출
            text = data.get('Meta', {}).get('passage', '')

            # 본문이 존재하면 저장
            if text:
                data_list.append({'category': category, 'text': text})

# 데이터프레임으로 변환 및 저장
df = pd.DataFrame(data_list)
df.to_csv('20per_data.csv', index=False, encoding='utf-8')
print(f"총 데이터 개수: {len(df)}")


총 데이터 개수: 40200


In [3]:
!pip install torch transformers pandas tqdm


In [4]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import PreTrainedTokenizerFast

# KoBART 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')

# 학습 데이터 로드
data = pd.read_csv('20per_data.csv')

# 데이터셋 클래스 정의
class KoBARTDataset(Dataset):
    def __init__(self, tokenizer, data, max_len=512):
        self.tokenizer = tokenizer
        self.data = data
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.loc[idx, 'text']  # 본문 데이터

        # 입력 텍스트와 출력 텍스트 생성
        inputs = self.tokenizer(
            text, max_length=self.max_len, truncation=True, return_tensors='pt', padding='max_length'
        )
        targets = self.tokenizer(
            text, max_length=self.max_len, truncation=True, return_tensors='pt', padding='max_length'
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': targets['input_ids'].squeeze()
        }

# 데이터셋 및 DataLoader 생성
dataset = KoBARTDataset(tokenizer, data)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


/home/metaai2/anaconda3/envs/dongsub/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [5]:
from transformers import BartForConditionalGeneration, AdamW
from tqdm import tqdm

# KoBART 모델 로드
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v2')

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습 설정
epochs = 3  # 학습 에포크 수


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/home/metaai2/anaconda3/envs/dongsub/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [6]:
import torch
from tqdm import tqdm

# GPU 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 중인 디바이스: {device}")

# 모델과 데이터를 GPU로 전송
model = model.to(device)

for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    for batch in tqdm(dataloader):
        # 배치 데이터를 GPU로 전송
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        # 모델 출력 및 손실 계산
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(dataloader)}")


사용 중인 디바이스: cuda


100%|██████████| 5025/5025 [11:06<00:00,  7.54it/s]


Epoch 1 Loss: 0.015812587825127063


100%|██████████| 5025/5025 [11:07<00:00,  7.53it/s]


Epoch 2 Loss: 0.0016382119948438552


100%|██████████| 5025/5025 [11:07<00:00,  7.53it/s]

Epoch 3 Loss: 0.0014195657006051653


In [7]:
model.save_pretrained('./kobart-finetuned-20per')
tokenizer.save_pretrained('./kobart-finetuned-20per')


/home/metaai2/anaconda3/envs/dongsub/lib/python3.12/site-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('./kobart-finetuned-20per/tokenizer_config.json',
 './kobart-finetuned-20per/special_tokens_map.json',
 './kobart-finetuned-20per/tokenizer.json')

In [7]:
import torch
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# KoBART 모델 및 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')  # KoBART 기본 모델
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v2').to(device)

# 텍스트 분리 함수
def split_text(text, max_length=512):
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]

# 요약 함수
def summarize(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding="max_length").to(device)
    summary_ids = model.generate(
        inputs['input_ids'], 
        max_length=100,  # 요약 최대 길이
        min_length=30,   # 요약 최소 길이
        length_penalty=1.5,  # 길이 패널티
        num_beams=5,  # 빔 서치 사용
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 테스트 텍스트
test_text = """
떡꼬치의 역사는 오래되었습니다. 떡꼬치는 고려시대부터 먹던 음식으로, 떡에 소시지, 치즈, 야채 등을 꽂아 구운 음식입니다. 떡꼬치는 간편하고 맛있게 즐길 수 있는 음식으로, 한국뿐만 아니라 세계적으로 인기가 있습니다.

떡꼬치의 재료는 떡, 소시지, 치즈, 야채 등이 있습니다. 떡은 쌀가루로 만든 음식으로, 떡꼬치의 주 재료입니다. 소시지는 돼지고기, 닭고기, 쇠고기 등으로 만든 음식으로, 떡꼬치에 풍미를 더합니다. 치즈는 우유로 만든 음식으로, 떡꼬치에 고소함을 더합니다. 야채는 당근, 파, 양파, 피망 등이 있습니다. 야채는 떡꼬치에 영양을 더합니다.

떡꼬치의 조리법은 간단합니다. 떡을 꼬치에 꽂고, 소시지, 치즈, 야채 등을 얹습니다. 그 다음, 떡꼬치를 오븐이나 그릴에 구워줍니다. 떡꼬치가 다 구워지면, 소금, 후추 등으로 간을 맞춥니다.

떡꼬치는 맛과 특징이 있습니다. 떡꼬치는 떡의 고소함과 소시지, 치즈, 야채의 맛이 어우러져 맛있습니다. 떡꼬치는 또한 간편하고 맛있게 즐길 수 있는 음식입니다.

떡꼬치는 영양성분이 있습니다. 떡꼬치는 떡, 소시지, 치즈, 야채 등이 들어있어 영양이 풍부합니다. 떡꼬치는 칼로리, 탄수화물, 단백질, 지방, 섬유질, 비타민, 미네랄 등이 풍부합니다.

떡꼬치는 문화적 의미가 있습니다. 떡꼬치는 한국의 대표적인 길거리 음식으로, 한국의 문화를 대표합니다. 떡꼬치는 또한 한국의 전통과 역사를 담고 있습니다.

떡꼬치는 한국의 대표적인 길거리 음식으로, 한국뿐만 아니라 세계적으로 인기가 있습니다. 떡꼬치는 간편하고 맛있게 즐길 수 있는 음식으로, 영양도 풍부합니다. 떡꼬치는 한국의 문화를 대표하는 음식으로, 한국의 전통과 역사를 담고 있습니다.
"""

# 텍스트 분리 및 요약
splitted_texts = split_text(test_text)
summaries = [summarize(part, model, tokenizer) for part in splitted_texts]

# 최종 요약 결과 출력
final_summary = " ".join(summaries)
print("\n최종 요약 결과:", final_summary)


Using device: cuda


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.



최종 요약 결과: 함께 치는 고려시대부터 먹던 음식으로, 떡에 소시지, 치즈, 야채 등을 꽂아 구운 음식입니다. 떡꼬치는 간편하고 맛있게 즐길 수 있는 음식으로, 한국뿐만 아니라 세계적으로 인기가 있습니다.

떡꼬치의 재료는 떡, 소시지, 치즈, 야채 등이 있습니다. 떡은 떡, 소시지, 치즈, 야채 등이 있습니다. 떡은 쌀가루로 만든 음식으로, 떡꼬치의 주 재료입니다. 떡꼬치의 주 재료입니다. 떡꼬치의 주 질,, 섬유질, 섬유질, 섬유질, 비타민, 비타민, 비타민, 미네랄 등이 풍부, 미네랄 등이 풍부 풍부, 미네랄 등이 풍부합니다.
떡꼬치는 문화적 의미가 있습니다.


In [9]:
#training data
import os
import json
import pandas as pd

# JSON 파일이 저장된 상위 디렉토리 경로
base_dir = r'/home/metaai2/byeonguk_work/I-live-A-commerce/TL1'  # JSON 파일이 포함된 폴더 경로
categories = ['fm_drama', 'culture', 'enter']  # 카테고리 폴더 리스트

# 데이터를 저장할 리스트
data_list = []

# JSON 데이터 처리
for category in categories:
    category_dir = os.path.join(base_dir, category, '3sent')  # 각 카테고리의 3sent 폴더 경로

    # 3sent 폴더 안의 모든 JSON 파일 처리
    for filename in os.listdir(category_dir):
        file_path = os.path.join(category_dir, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)

            # 필요한 정보 추출
            doc_type = json_data.get('Meta', {}).get('doc_type', '')
            doc_id = json_data.get('Meta', {}).get('doc_id', '')
            doc_origin = json_data.get('Meta', {}).get('doc_origin', '')
            passage = json_data.get('Meta', {}).get('passage', '')
            summary = (
                json_data.get('Annotation', {}).get('Summary1', '') or
                json_data.get('Annotation', {}).get('Summary2', '') or
                json_data.get('Annotation', {}).get('Summary3', '')
            )

            # 리스트에 저장
            data_list.append({
                'type': doc_type,
                'id': doc_id,
                'origin': doc_origin,
                'passage': passage,
                'summary': summary
            })

# 데이터프레임으로 변환
df = pd.DataFrame(data_list)

# CSV 파일로 저장
output_file = 'training_data.csv'
df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"CSV 파일로 저장 완료: {output_file}")


CSV 파일로 저장 완료: training_data.csv


In [11]:
#Validation data
import os
import json
import pandas as pd

# JSON 파일이 저장된 상위 디렉토리 경로
base_dir = r'/home/metaai2/byeonguk_work/I-live-A-commerce/TL1'  # JSON 파일이 포함된 폴더 경로
categories = ['fs_drama', 'c_event']  # 카테고리 폴더 리스트

# 데이터를 저장할 리스트
data_list = []

# JSON 데이터 처리
for category in categories:
    category_dir = os.path.join(base_dir, category, '3sent')  # 각 카테고리의 3sent 폴더 경로

    # 3sent 폴더 안의 모든 JSON 파일 처리
    for filename in os.listdir(category_dir):
        file_path = os.path.join(category_dir, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)

            # 필요한 정보 추출
            doc_type = json_data.get('Meta', {}).get('doc_type', '')
            doc_id = json_data.get('Meta', {}).get('doc_id', '')
            doc_origin = json_data.get('Meta', {}).get('doc_origin', '')
            passage = json_data.get('Meta', {}).get('passage', '')
            summary = (
                json_data.get('Annotation', {}).get('Summary1', '') or
                json_data.get('Annotation', {}).get('Summary2', '') or
                json_data.get('Annotation', {}).get('Summary3', '')
            )

            # 리스트에 저장
            data_list.append({
                'type': doc_type,
                'id': doc_id,
                'origin': doc_origin,
                'passage': passage,
                'summary': summary
            })

# 데이터프레임으로 변환
df = pd.DataFrame(data_list)

# CSV 파일로 저장
output_file = 'Validation_data.csv'
df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"CSV 파일로 저장 완료: {output_file}")


CSV 파일로 저장 완료: Validation_data.csv


In [12]:
#test data
import os
import json
import pandas as pd

# JSON 파일이 저장된 상위 디렉토리 경로
base_dir = r'/home/metaai2/byeonguk_work/I-live-A-commerce/TL1'  # JSON 파일이 포함된 폴더 경로
categories = ['history']  # 카테고리 폴더 리스트

# 데이터를 저장할 리스트
data_list = []

# JSON 데이터 처리
for category in categories:
    category_dir = os.path.join(base_dir, category, '3sent')  # 각 카테고리의 3sent 폴더 경로

    # 3sent 폴더 안의 모든 JSON 파일 처리
    for filename in os.listdir(category_dir):
        file_path = os.path.join(category_dir, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)

            # 필요한 정보 추출
            doc_type = json_data.get('Meta', {}).get('doc_type', '')
            doc_id = json_data.get('Meta', {}).get('doc_id', '')
            doc_origin = json_data.get('Meta', {}).get('doc_origin', '')
            passage = json_data.get('Meta', {}).get('passage', '')
            summary = (
                json_data.get('Annotation', {}).get('Summary1', '') or
                json_data.get('Annotation', {}).get('Summary2', '') or
                json_data.get('Annotation', {}).get('Summary3', '')
            )

            # 리스트에 저장
            data_list.append({
                'type': doc_type,
                'id': doc_id,
                'origin': doc_origin,
                'passage': passage,
                'summary': summary
            })

# 데이터프레임으로 변환
df = pd.DataFrame(data_list)

# CSV 파일로 저장
output_file = 'test_data.csv'
df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"CSV 파일로 저장 완료: {output_file}")


CSV 파일로 저장 완료: test_data.csv


In [18]:
#csv는 구분자가 콤마(,)이고, tsv는 구분자가 탭(\t)
#train_data
import pandas as pd
import shutil
path = '/home/metaai2/byeonguk_work/I-live-A-commerce/training_data.csv'
df = pd.read_csv(path, sep = ",", engine='python', encoding = "utf-8")
df.dropna(axis=0)
df.to_csv('train.tsv', sep='\t', encoding="utf-8", index=False)

# 원본 파일 경로
source_file = '/home/metaai2/byeonguk_work/I-live-A-commerce/train.tsv'
# 이동할 위치의 폴더 경로
destination_folder = '/home/metaai2/byeonguk_work/I-live-A-commerce/TL1/data/'
# 파일 이동
shutil.move(source_file, destination_folder)

Error: Destination path '/home/metaai2/byeonguk_work/I-live-A-commerce/TL1/data/train.tsv' already exists

In [17]:
#csv는 구분자가 콤마(,)이고, tsv는 구분자가 탭(\t)
#train_data
import pandas as pd
import shutil
path = '/home/metaai2/byeonguk_work/I-live-A-commerce/Validation_data.csv'
df = pd.read_csv(path, sep = ",", engine='python', encoding = "utf-8")
df.dropna(axis=0)
df.to_csv('validation.tsv', sep='\t', encoding="utf-8", index=False)

# 원본 파일 경로
source_file = '/home/metaai2/byeonguk_work/I-live-A-commerce/validation.tsv'
# 이동할 위치의 폴더 경로
destination_folder = '/home/metaai2/byeonguk_work/I-live-A-commerce/TL1/data/'
# 파일 이동
shutil.move(source_file, destination_folder)

'/home/metaai2/byeonguk_work/I-live-A-commerce/TL1/data/validation.tsv'

In [19]:
#csv는 구분자가 콤마(,)이고, tsv는 구분자가 탭(\t)
#train_data
import pandas as pd
import shutil
path = '/home/metaai2/byeonguk_work/I-live-A-commerce/test_data.csv'
df = pd.read_csv(path, sep = ",", engine='python', encoding = "utf-8")
df.dropna(axis=0)
df.to_csv('test.tsv', sep='\t', encoding="utf-8", index=False)

# 원본 파일 경로
source_file = '/home/metaai2/byeonguk_work/I-live-A-commerce/test.tsv'
# 이동할 위치의 폴더 경로
destination_folder = '/home/metaai2/byeonguk_work/I-live-A-commerce/TL1/data/'
# 파일 이동
shutil.move(source_file, destination_folder)

'/home/metaai2/byeonguk_work/I-live-A-commerce/TL1/data/test.tsv'

In [20]:
!pip install transformers==3.5.1
!pip install loguru
!pip install lightning==2.0.8
!pip install tokenizers==0.13.3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      Package sentencepiece was not found in the pkg-config search path.
      Perhaps you should add the directory containing `sentencepiece.pc'
      to the PKG_CONFIG_PATH environment variable
      No package 'sentencepiece' found
      Failed to find sentencepiece pkgconfig
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached arrow-1.3.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached fastapi-0.115.6-py3-none-any.whl.metadata (27 kB)
  Using cached lightning_utilities-0.11.9-py3-none-any.whl.metadata (5.2 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
  Using cached starlette-0.41.3-py3-none-any.whl.metadata (6.0 kB)
  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.7 MB/s eta 0:00:00
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached fastapi-0.115.6-py3-none-any.whl (94 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 78.3 MB/s eta 0:00:00
Using cached starlette-0.41.3-py3-none-any.whl (73 kB)
   ━

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [49 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build/lib.linux-x86_64-cpython-312/tokenizers
      copying py_src/tokenizers/__init__.py -> build/lib.linux-x86_64-cpython-312/tokenizers
      creating build/lib.linux-x86_64-cpython-312/tokenizers/models
      copying py_src/tokenizers/models/__init__.py -> build/lib.linux-x86_64-cpython-312/tokenizers/models
      creating build/lib.linux-x86_64-cpython-312/tokenizers/decoders
      copying py_src/tokenizers/decoders/__init__.py -> build/lib.linux-x86_64-cpython-312/tokenizers/decoders
      creating build/lib.linux-x86_64-cpython-312/tokenizers/normalizers
      copying py_src/tokenizers/normalizers/__init_

In [39]:
!pip install wandb==0.15.9
!pip install tensorboard


  Using cached wandb-0.15.9-py3-none-any.whl.metadata (8.3 kB)
  Using cached pathtools-0.1.2.tar.gz (11 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-7oxopsde/pathtools_79489bf72a50404681a187e1c4d17807/setup.py", line 25, in <module>
          import imp
      ModuleNotFoundError: No module named 'imp'
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [1]:
def foo():
    pass

In [2]:
!CUDA_VISIBLE_DEVICES=1 python train.py --gradient_clip_val 1.0 \
                                       --max_epochs 50 \
                                       --checkpoint checkpoint \
                                       --accelerator gpu \
                                       --num_gpus 1 \
                                       --batch_size 4 \
                                       --num_workers 4

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
2025-01-21 19:37:20.765 | INFO     | __main__:<module>:72 - Namespace(train_file='/home/metaai2/byeonguk_work/I-live-A-commerce/TL1/data/train.tsv', test_file='/home/metaai2/byeonguk_work/I-live-A-commerce/TL1/data/test.tsv', batch_size=4, checkpoint='checkpoint', max_len=512, max_epochs=100, lr=3e-05, accelerator='gpu', num_gpus=1, gradient_clip_val=1.0, num_workers=4)
Loaded dataset with columns: ['type', 'id', 'origin', 'passage', 'summary']
Loaded dataset with columns: ['type', 'id', 'origin', 'passage', 'summary']
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
GPU available: True

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration 
!python get_model_binary.py --model_binary '/content/drive/MyDrive/KoBART-summarization-main/KoBART-summarization-main/checkpoint/summarization_final/epoch=99-val_loss=0.000.ckpt'

# 모델 바이너리 파일 경로
model_binary_path = '/content/drive/MyDrive/KoBART-summarization-main/KoBART-summarization-main/kobart_summary'

# KoBART 모델 및 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')
model = BartForConditionalGeneration.from_pretrained(model_binary_path)

In [ ]:
# 입력 텍스트
input_text =  """
광화문(光化門)은 서울특별시 종로구의 조선왕조 법궁인 경복궁의 남쪽에 있는 정문이다. "임금의 큰 덕(德)이 온 나라를 비춘다"는 의미이다. 1395년에 세워졌으며, 2층 누각 구조로 되어 있다. 경복궁의 정전인 근정전으로 가기 위해 지나야 하는 문 3개 중에서 첫째로 마주하는 문이며, 둘째는 흥례문, 셋째는 근정문이다. 광화문 앞에는 지금은 도로 건설로 사라진 월대가 자리잡고 있었으며, 양쪽에는 한 쌍의 해태 조각상이 자리잡고 있다. 광화문의 석축부에는 세 개의 홍예문(虹霓門, 아치문)이 있다. 가운데 문은 임금이 다니던 문이고, 나머지 좌우의 문은 신하들이 다니던 문이었는데, 왼쪽 문은 무신이, 오른쪽 문은 문신이 출입했다. 광화문의 가운데 문 천장에는 주작이 그려져 있고, 왼쪽 문에는 거북이가, 오른쪽 문에는 천마가 그려져 있다.
"""
# 입력 텍스트를 토큰화하여 인코딩
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)

# 모델을 사용하여 요약 생성
summary_ids = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

# 요약 결과 디코딩
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 요약 출력
print("요약 텍스트:", summary_text)